In [1]:
import pandas as pd
import re

In [2]:
# === 0. Đọc dữ liệu ===
df = pd.read_csv('D:\\NAM 3\\Ki II\\Đồ án II\\dataset\\CellphoneS.csv', encoding='utf-8')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      388 non-null    object 
 1   web-scraper-start-url  388 non-null    object 
 2   link                   388 non-null    object 
 3   link-href              388 non-null    object 
 4   title                  388 non-null    object 
 5   price                  315 non-null    object 
 6   ratings                249 non-null    object 
 7   num_rates              249 non-null    float64
 8   screen_size            381 non-null    object 
 9   refresh_rate           286 non-null    object 
 10  chipset                374 non-null    object 
 11  GPU                    287 non-null    object 
 12  sim_slot               350 non-null    object 
 13  RAM                    351 non-null    object 
 14  internal_storage       373 non-null    object 
 15  batter

In [3]:
# === 1. Xóa các cột không cần thiết nếu có ===
columns_to_drop = [
    'web-scraper-order', 'web-scraper-start-url',
    'link', 'link-href', 'type'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

In [4]:
# === 2. Xử lý cột giá (price) ===
df['price'] = df['price'].astype(str).str.replace(r'[^\d]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [5]:
# === 3. Hàm trích số từ chuỗi ===
def extract_number(x):
    if pd.isnull(x): return None
    x = str(x).replace(',', '')
    match = re.search(r'(\d+)', x)
    return int(match.group(1)) if match else None

In [6]:
# === 4. Áp dụng cho RAM, internal_storage, battery ===
for col in ['RAM', 'internal_storage', 'battery']:
    if col in df.columns:
        df[col] = df[col].apply(extract_number)

In [7]:
# === 5. Xử lý ratings: '4.5/5' → 4.5
if 'ratings' in df.columns:
    df['ratings'] = df['ratings'].astype(str).str.extract(r'([\d.]+)')
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')

In [8]:
# === 6. Xử lý refresh_rate: trích số Hz từ chuỗi
def extract_refresh_rate(x):
    match = re.search(r'(\d{2,3})\s*Hz', str(x))
    if match:
        return int(match.group(1))
    return None

In [9]:
if 'refresh_rate' in df.columns:
    df['refresh_rate'] = df['refresh_rate'].apply(extract_refresh_rate)

In [10]:
# === 7. Loại bỏ dòng không có giá
df = df.dropna(subset=['price'])

In [11]:
# === 8. Điền median cho các cột số ===
numeric_cols = ['num_rates', 'RAM', 'internal_storage', 'battery', 'refresh_rate']
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

In [12]:
# === 9. Điền mode cho các cột phân loại ===
categorical_cols = ['ratings', 'screen_size', 'chipset', 'GPU', 'sim_slot', 'operating_system']
for col in categorical_cols:
    if col in df.columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])

In [13]:
def simplify_os(os_name):
    os_name = os_name.lower()
    if "android" in os_name:
        return "Android"
    elif "ios" in os_name:
        return "iOS"
    elif "s30" in os_name:
        return "S30+"
    else:
        return "Khác"

df['operating_system'] = df['operating_system'].apply(simplify_os)


In [16]:
# === 10. Xuất file Excel
df.to_csv('du_lieu_da_xu_ly.csv', index=False,encoding='utf-8-sig')

In [15]:
# === 11. In kiểm tra
print("✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'")
print(df.head())
print("\n⚠️ Kiểm tra thiếu giá trị:")
print(df.isnull().sum())

✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'
                         title       price  ratings  num_rates  screen_size  \
1  ASUS ROG Phone 6 12GB 256GB  14490000.0      5.0       20.0  6.78 inches   
2          BENCO V91 4GB 128GB   2690000.0      5.0        1.0  6.56 inches   
3       Benco S1 Pro 8GB 256GB   4290000.0      5.0       10.0   6.8 inches   
5                 INOI 288S 4G   1000000.0      5.0        1.0   2.0 inches   
6      INOI Note 13s 8GB 256GB   2790000.0      5.0        2.0  6.95 inches   

   refresh_rate                              chipset           GPU  \
1         120.0  Qualcomm ® Snapdragon ® 8+ thế hệ 1  Adreno ™ 730   
2          90.0                          Unisoc T606    Adreno 610   
3         120.0                          Unisoc T616      Mali-G57   
5         120.0                        Apple A18 Pro    Adreno 610   
6         120.0                          Unisoc T606    Adreno 610   

           sim_slot   RAM  internal_storage  